## The Advanced Computing Center for Research and Education

The Advanced Computing Center for Research and Education (ACCRE) is a computer cluster serving the high-performance computing needs of research for Vanderbilt University. In this data question, you will be analyzing data on jobs run on ACCRE's hardware.

The computing resources in the ACCRE cluster are divided into nodes. Each node is equipped with some number of CPUs, and some of the nodes contain GPUs (graphical processing units).  You have been provided a dataset, **accre-gpu-jobs-2022.csv**, which contains information on jobs submitted to ACCRE's GPU partitions. Traditionally GPUs were designed to power video games to perform calculations quickly. Because of the nature of their design, GPUs are being used more and more for non-graphics applications as well (e.g. for deep learning applications, molecular dynamics, image processing, and much more).

Recently, more researchers are performing work that requires GPU acceleration. Your task in this project is to analyze the GPU jobs that have been run on ACCRE over the last few years in order to better understand what needs to be provisioned moving forward. 

ACCRE has three scheduler partitions that groups can get access to which consist of machines each with 4 GPU cards. Each partition is named after the nvidia processor generation that it contains, from oldest to newest: maxwell, pascal, and turing. GPU nodes are requested per-GPU, and users can request up to 4 GPUs per node. For each GPU the user is allocated up to one quarter of the machine's RAM and CPU cores. All of the current GPU-accelerated hardware is also connected to a special high-speed RoCE (RDMA over converged ethernet) network allowing for memory sharing on large-scale multi-GPU jobs, so users can also request multiple servers for analysis that requires more than four GPU cards.

In [1]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
accre_df = pd.read_csv('../data/accre-gpu-jobs-2022-v2.csv', on_bad_lines='warn')

b'Skipping line 124084: expected 12 fields, saw 13\nSkipping line 124809: expected 12 fields, saw 13\n'


there are 2 bad lines, we'll ignore them for now...

In [3]:
accre_df.shape

(336950, 12)

In [4]:
accre_df.head()

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED


Let's check how many Used Memory are 0 (because they ran less than 2 min)

In [5]:
accre_df.groupby('USEDMEM').count()

,JOBID,ACCOUNT,USER,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
USEDMEM,,,,,,,,,,,
0,58747,58747,58747,58747,58747,58747,58747,58747,58747,58747,58747
0.00M,17,17,17,17,17,17,17,17,17,17,17
0.01M,15,15,15,15,15,15,15,15,15,15,15
0.02M,3,3,3,3,3,3,3,3,3,3,3
0.03M,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...
9990.15M,1,1,1,1,1,1,1,1,1,1,1
9990.72M,1,1,1,1,1,1,1,1,1,1,1
9991.41M,1,1,1,1,1,1,1,1,1,1,1


In [6]:
# accre_df[accre_df['USEDMEM'].str.match(r'0.?0?0?M?') == True]

In [7]:
accre_df[(accre_df['USEDMEM'] == '0') | (accre_df['USEDMEM'] == '0.00M')]

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
6,32880722,glasshouse,brady,0,3-12:00:00,3-12:00:07,1,1,1,pascal,0:0,COMPLETED
8,32880758,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,CANCELLED by 9201
9,32880759,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,CANCELLED by 9201
10,32880770,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,CANCELLED by 9201
...,...,...,...,...,...,...,...,...,...,...,...,...
336944,46754104,glasshouse,milford,0,03:00:00,00:22:57,1,8,4,pascal,0:0,RUNNING
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING


In [8]:
accre_noM = accre_df[(accre_df['USEDMEM'] != '0') & (accre_df['USEDMEM'] != '0.00M')]

Just cleared out all the 0 (or 0.00M) jobs, that either had some problems and didn't use any memory, or, they ran less than 2 min. That was around 58764 of jobs! Out of 336950 jobs.

In [9]:
accre_df.groupby('STATE').count()

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE
STATE,,,,,,,,,,,
CANCELLED,947,947,947,947,947,947,947,947,947,947,947
CANCELLED by 0,1,1,1,1,1,1,1,1,1,1,1
CANCELLED by 111694,11,11,11,11,11,11,11,11,11,11,11
CANCELLED by 133418,23,23,23,23,23,23,23,23,23,23,23
CANCELLED by 153500,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...
COMPLETED,283333,283333,283333,283333,283333,283333,283333,283333,283333,283333,283333
FAILED,440,440,440,440,440,440,440,440,440,440,440
OUT_OF_MEMORY,3,3,3,3,3,3,3,3,3,3,3


Interestingly, besides the COMPLETED being 0:0 and pending as well, the CANCELLED jobs are also 0:0

In [10]:
accre_noM[accre_noM['STATE'] != 'COMPLETED'].groupby('EXITCODE').count()

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,STATE
EXITCODE,,,,,,,,,,,
0:0,12,12,12,12,12,12,12,12,12,12,12
0:125,3,3,3,3,3,3,3,3,3,3,3
0:15,130,130,130,130,130,130,130,130,130,130,130
0:6,10,10,10,10,10,10,10,10,10,10,10
0:7,1,1,1,1,1,1,1,1,1,1,1
0:9,185,185,185,185,185,185,185,185,185,185,185
126:0,6,6,6,6,6,6,6,6,6,6,6
127:0,6,6,6,6,6,6,6,6,6,6,6
13:0,5,5,5,5,5,5,5,5,5,5,5


Figuring out codes, because ... why not??

- observation: it appears than when the error was on the cluster's side, it was mostly due to a cancelled job, and 3x for out of memory. On the user's side, mostly failed

In [11]:
accre_noM.groupby(['EXITCODE','STATE']).count()

JOBID  ACCOUNT    USER  USEDMEM  REQTIME  \
EXITCODE STATE                                                            
0:0      CANCELLED                12       12      12       12       12   
         COMPLETED            277475   277475  277475   277475   277475   
0:125    OUT_OF_MEMORY             3        3       3        3        3   
0:15     CANCELLED               130      130     130      130      130   
0:6      CANCELLED                10       10      10       10       10   
0:7      CANCELLED                 1        1       1        1        1   
0:9      CANCELLED               135      135     135      135      135   
         CANCELLED by 0            1        1       1        1        1   
         CANCELLED by 159541       1        1       1        1        1   
         CANCELLED by 161909       4        4       4        4        4   
         CANCELLED by 196703       1        1       1        1        1   
         CANCELLED by 46207        2        2       2        2        2   
         CANCELLED by 505355      11       11      11       11       11   
         CANCELLED by 649321      17       17      17       17       17   
         CANCELLED by 756088       1        1       1        1        1   
         CANCELLED by 785491       6        6       6        6        6   
         CANCELLED by 859439       1        1       1        1        1   
         CANCELLED by 885613       5        5       5        5        5   
126:0    CANCELLED                 4        4       4        4        4   
         FAILED                    2        2       2        2        2   
127:0    FAILED                    6        6       6        6        6   
13:0     FAILED                    5        5       5        5        5   
16:0     FAILED                    3        3       3        3        3   
1:0      CANCELLED                 3        3       3        3        3   
         FAILED                  321      321     321      321      321   
2:0      FAILED                   16       16      16       16       16   
33:0     FAILED                    2        2       2        2        2   
7:0      CANCELLED                 1        1       1        1        1   
         FAILED                    6        6       6        6        6   
9:0      FAILED                    1        1       1        1        1   

                              USEDTIME   NODES    CPUS    GPUS  PARTITION  
EXITCODE STATE                                                             
0:0      CANCELLED                  12      12      12      12         12  
         COMPLETED              277475  277475  277475  277475     277475  
0:125    OUT_OF_MEMORY               3       3       3       3          3  
0:15     CANCELLED                 130     130     130     130        130  
0:6      CANCELLED                  10      10      10      10         10  
0:7      CANCELLED                   1       1       1       1          1  
0:9      CANCELLED                 135     135     135     135        135  
         CANCELLED by 0              1       1       1       1          1  
         CANCELLED by 159541         1       1       1       1          1  
         CANCELLED by 161909         4       4       4       4          4  
         CANCELLED by 196703         1       1       1       1          1  
         CANCELLED by 46207          2       2       2       2          2  
         CANCELLED by 505355        11      11      11      11         11  
         CANCELLED by 649321        17      17      17      17         17  
         CANCELLED by 756088         1       1       1       1          1  
         CANCELLED by 785491         6       6       6       6          6  
         CANCELLED by 859439         1       1       1       1          1  
         CANCELLED by 885613         5       5       5       5          5  
126:0    CANCELLED                   4       4       4       4          4  
         FAILED                   

In [12]:
accre_noM[accre_noM['STATE'] == 'OUT_OF_MEMORY']

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
332639,46247498,glasshouse,tommie,138663.87M,5-00:00:00,18:01:08,2,24,8,turing,0:125,OUT_OF_MEMORY
333434,46329470,pasilla,venus,7.19M,00:30:00,00:05:05,1,1,1,turing,0:125,OUT_OF_MEMORY
334513,46514679,glasshouse,arnoldo,24221.36M,04:00:00,01:11:23,1,6,1,turing,0:125,OUT_OF_MEMORY


kind of interesting that a job failed with only 7.19M usage and 5 minutes of run time.

In [13]:
accre_noM.groupby('NODES').count()

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,CPUS,GPUS,PARTITION,EXITCODE,STATE
NODES,,,,,,,,,,,
1,277454,277454,277454,277454,277454,277454,277454,277454,277454,277454,277454
2,252,252,252,252,252,252,252,252,252,252,252
3,336,336,336,336,336,336,336,336,336,336,336
4,66,66,66,66,66,66,66,66,66,66,66
5,56,56,56,56,56,56,56,56,56,56,56
6,1,1,1,1,1,1,1,1,1,1,1
7,12,12,12,12,12,12,12,12,12,12,12
8,1,1,1,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,1


In [14]:
accre_noM[accre_noM['NODES'] == 15]

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
62122,37020263,pasilla,dola,2.53M,00:20:00,00:00:03,15,120,60,pascal,0:0,COMPLETED
270301,44976228,pasilla,dola,7.41M,00:20:00,00:00:02,15,120,60,pascal,0:0,COMPLETED


The main objectives of this project are to examine the following questions:
1. What is the distribution of per-GPU main memory usage over all runtime-weighed jobs in each partition? Knowing this will help ACCRE to understand our users memory needs for future hardware purchases.

In [19]:
accre_noM['USEDMEM'] = accre_noM['USEDMEM'].str.replace('M','')

/var/folders/3t/x8rz35tn765cm0bkf_n4ydlh0000gn/T/ipykernel_78324/2007413859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accre_noM['USEDMEM'] = accre_noM['USEDMEM'].str.replace('M','')


In [61]:
# I can think of two different approaches - either with regex, or, 
# with length, to get the time values (days, hours, mins, secs) and convert them to seconds. 
# I was trying to figure out how to approach the time ... option 1:
# re.findall(r'(.?)-?(..):(..):(..)', accre_noM['USEDTIME'][327])
# re.findall(r'(.?)-?(..):(..):(..)', accre_noM['USEDTIME'][4])

# option 2 ( using len) and then some type of list comprehension(not finished):
# len(accre_noM['USEDTIME'][4])
#accre_noM['USEDTIME_SEC'] = (
#     [float(x[0]) * 86400 + float(x[]) 
#      if len(x) == 10 
#      else None 
#      for x in accre_noM['USEDTIME']]    
#)
# but there is, of course, a dataframe function!

In [65]:
pd.to_datetime(accre_noM['USEDTIME'])

ParserError: Unknown string format: 1-08:38:34


2. What is the distribution of the number of GPUs in each job (runtime-weighted) for each partition? What fraction of runtime-weighted and GPU-weighted jobs are using more than 4 GPUs and thus probably using the RoCE networking? Is this fraction different for each partition?
3. What is the total runtime usage per-gpu (i.e. multiply runtime by the number of gpus) in each of the 3 partitions over the last year?
4. What is the distribution of different groups and users accessing each partition? In each partition, who are the top users, and do they represent a majority of the runtime-weighted jobs on the partition?
5. Currently there is a 5 day limit on runtime for GPU jobs, although some users have been asking for extensions. What is the distribution of requested runtime and actual runtime on jobs on each partition? Do users really need more time, or are they simply always requesting the maximum?